In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

In [2]:
# Creating a larger dummy dataset
num_samples = 10000  # Increased dataset size
data = {
    "Appetite": np.random.choice(["Normal", "Increased", "Decreased", "No appetite"], num_samples),
    "Energy_Level": np.random.choice(["Normal", "Hyperactive"], num_samples),
    "Water_Intake": np.random.choice(["Normal amount", "Drinking more than usual", "Drinking less than usual"], num_samples),
    "Vomiting": np.random.choice(["No", "Occasionally", "Frequently"], num_samples),
    "Diarrhea": np.random.choice(["No", "Mild", "Severe"], num_samples),
    "Coughing": np.random.choice(["No", "Occasionally", "Frequently", "Lethargic"], num_samples),
    "Breathing": np.random.choice(["Normal", "Rapid", "Labored"], num_samples),
    "Skin_Coat_Changes": np.random.choice(["No changes", "Dry/flaky skin", "Rash or redness", "Hair loss"], num_samples),
    "Urinating": np.random.choice(["No changes", "Increased frequency", "Decreased frequency", "Blood in urine"], num_samples),
    "Behavior_Changes": np.random.choice(["No changes", "More aggressive", "More withdrawn", "Confused or disoriented"], num_samples),
    "Body_Temperature": np.random.uniform(35.0, 42.0, num_samples),  # Temperature between 35 and 42 degrees
    "Disease": np.random.choice(["Healthy", "Diabetes", "Kidney Disease", "Infection", "Gastroenteritis", "Heart Disease"], num_samples)
}

df = pd.DataFrame(data)

In [3]:
# Encoding categorical variables
label_encoders = {}
for column in df.columns[:-2]:  # Exclude target variable and numerical temperature
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Scaling Body Temperature
scaler = StandardScaler()
df["Body_Temperature"] = scaler.fit_transform(df[["Body_Temperature"]])

# Splitting dataset
X = df.drop(columns=["Disease"])
y = df["Disease"]

# Handling class imbalance with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Training Random Forest Model
rf = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=42)
rf.fit(X_train, y_train)

# Evaluating the model
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Model Accuracy: {accuracy:.2f}")

Random Forest Model Accuracy: 0.22


In [5]:
# User input for diagnosis
print("Answer the following questions about your pet:")
user_input = {}
questions = {
    "Appetite": ["Normal", "Increased", "Decreased", "No appetite"],
    "Energy_Level": ["Normal", "Hyperactive"],
    "Water_Intake": ["Normal amount", "Drinking more than usual", "Drinking less than usual"],
    "Vomiting": ["No", "Occasionally", "Frequently"],
    "Diarrhea": ["No", "Mild", "Severe"],
    "Coughing": ["No", "Occasionally", "Frequently", "Lethargic"],
    "Breathing": ["Normal", "Rapid", "Labored"],
    "Skin_Coat_Changes": ["No changes", "Dry/flaky skin", "Rash or redness", "Hair loss"],
    "Urinating": ["No changes", "Increased frequency", "Decreased frequency", "Blood in urine"],
    "Behavior_Changes": ["No changes", "More aggressive", "More withdrawn", "Confused or disoriented"]
}

for key, options in questions.items():
    print(f"{key}: {options}")
    choice = input("Enter your choice: ")
    
    if key in label_encoders:
        choice = label_encoders[key].transform([choice])[0]
    
    user_input[key] = choice

# Asking for body temperature
while True:
    try:
        temp = float(input("Enter your pet's body temperature (35.0 - 42.0): "))
        if 35.0 <= temp <= 42.0:
            user_input["Body_Temperature"] = scaler.transform([[temp]])[0][0]
            break
        else:
            print("Please enter a valid temperature between 35.0 and 42.0.")
    except ValueError:
        print("Invalid input. Please enter a numerical value.")

# Convert user input into a DataFrame
user_df = pd.DataFrame([user_input], columns=X_train.columns)


# Predict disease
prediction = rf.predict(user_df)
print(f"Predicted Disease: {prediction[0]}")

Answer the following questions about your pet:
Appetite: ['Normal', 'Increased', 'Decreased', 'No appetite']
Enter your choice: Normal
Energy_Level: ['Normal', 'Hyperactive']
Enter your choice: Hyperactive
Water_Intake: ['Normal amount', 'Drinking more than usual', 'Drinking less than usual']
Enter your choice: Normal amount
Vomiting: ['No', 'Occasionally', 'Frequently']
Enter your choice: Occasionally
Diarrhea: ['No', 'Mild', 'Severe']
Enter your choice: Mild
Coughing: ['No', 'Occasionally', 'Frequently', 'Lethargic']
Enter your choice: No
Breathing: ['Normal', 'Rapid', 'Labored']
Enter your choice: Rapid
Skin_Coat_Changes: ['No changes', 'Dry/flaky skin', 'Rash or redness', 'Hair loss']
Enter your choice: No changes
Urinating: ['No changes', 'Increased frequency', 'Decreased frequency', 'Blood in urine']
Enter your choice: No changes
Behavior_Changes: ['No changes', 'More aggressive', 'More withdrawn', 'Confused or disoriented']
Enter your choice: More aggressive
Enter your pet's bod

E:\Users\Wrishita Paul\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
